# 1. 开始迭代：bach_size与epoches

## 1.1 为什么要有小批量

在实现一轮梯度下降后，只要在梯度下降的代码外面加上一层循环，就可以顺利实现迭代多次的梯度下降了。但在那之前，还有另外一个问题。为了提升梯度下降的速度，我们在使用了动量法，同时，我们也要在使用的数据上下功夫。

在深度学习的世界中，神经网络的训练对象往往是图像、文字、语音、视频等非结构化数据。这些数据的特点就是特征张量一般都是大量高维的数据。比如在深度学习教材中总是被用来做例子的入门级数据MINIST,其训练集的结构为（60000， 784）。对比机器学习的入门级数据鸢尾花（其结构为（150，4）），两者完全不在一个量级上。在深度学习中，如果梯度下降的每次迭代都使用全部数据，将会非常耗费计算资源，且样本量越大，计算开销越高。虽然PyTorch被设计成天生能够处理巨量数据，但我们还是需要在数据量这一点上下功夫。

小批量随机梯度下降（mini-bach stochastic gradient descent）是深度学习入门级的优化算法（梯度下降是入门级之下的），其求解与迭代流程与传统梯度下降（GD）基本一致，不过二者在迭代权重时使用的数据这一点上存在巨大的不同。传统梯度下降在每次进行权重迭代（即循环）时都使用全部数据，每次迭代所使用的数据也都一致。而mini-bach SGD是每次迭代前都会从整体采样一批固定数目的样本批次组成（bach）B，并使用B中的样本进行梯度计算，以减少样本量。

为什么会选择mini-bach SGD作为神经网络的入门级优化算法呢？有两个比较主流的原因。第一个是，比起传统梯度下降，mini-bach SGD更可能找到全局最小值。

![Alt text](image-21.png)

尽可能找到全局最优一直都是优化算法的目标，为什么说mini-bach SGD更容易找到全局最优呢？

![Alt text](image-22.png)

## 1.2 batch_size与epoches

在min-bach SGD中，我们选择的批量batch含有的样本数被称为 batch_size，批量尺寸，这个尺寸一定是小雨数据量的某个正整数值。每次迭代之前，我们需要从数据集中抽取batch_size个数据用于训练。

在普通梯度下降中，因为没有抽样，所以每次迭代就会将所有数据都使用一次，迭代了t次时，算法就将数据学习了t次。可以想象，对同样的数据，算法学习得越多，也就应当对数据的状况理解得越深，也就学得越好。然而，并不是对一个数据学习越多越好，毕竟学习得越多，训练时间就越长，同时，我们能够收集到的数据只是“样本”，并不能代表真实世界的客观情况。例如，我们从几万张猫与狗的照片中学到的内容，并不一定能够适用于全世界所有的猫和狗。如果我们的照片中猫咪都是有毛的，那神经网络对数据的学习程度越深，它就越有可能认不出无毛猫。因此，我们虽然希望算法对数据了解很深，但我们也希望算法不要变成“书呆子”，要保留一些灵活性（保留一些泛化能力）。关于这一点，我们之后会详细展开说明，但大家现在需要知道的是，算法对同样的数据进行学习的次数并不是越多越好

在mini-bach SGD中，因为每次迭代时只使用了一小部分数据，所以它迭代次数并不代表全体数据一共被学习了多少次。所以我们需要另一个重要概念：epoch，来定义全体数据一共被学习了多少次：

    重要概念：epoch
    epoch是衡量训练数据被使用次数的单位，一个epoch表示优化算法将全部训练数据使用了一次。它与梯度下降中的迭代次数有非常深的关系，我们常使用“完成一个epoch需要n次迭代”这样的语言。

假设一个数据集总共有m个样本，我们选择的bach_size是 $ N_{B} $,即每次迭代时都使用  $ N_{B} $个样本，则一个epoch所需的迭代次数的计算公式如下：

完成一个epoch所需要的迭代次数 n = m/ $ N_{B} $

在深度学习中，我们常常定义num_epoches作为梯度下降的最外层循环，bach_size作为内层循环。有时候，我们希望数据被多学习几次，来增加模型对数据的理解。有时候，我们会控制模型对数据的训练，总而言之，我们会使用epoch和batch_size来控制训练的节奏。接下来，我们就用代码来实现一下：

In [ ]:
# epoch = 60 让神经网络学习60次数据
# 把全部数据X分为10个batch
for epochs in range(epoch):
    for batch in range(batch):

 ## 1.3 TensorDataset与DataLoader

 要使用小批量随机梯度下降，我们就需要对数据进行采样，分割等操作。在PyTorch中，操作数据所需要使用的模块是torch.utils，其中utils.data类下面有大量用来执行数据预处理的工具。在MBSGD中，我们需要将数据划分为许多特征张量+对应标签的形式，因此最开始我们要将数据的特征张量与标签打包成一个对象。之前我们提到过，深度学习中的特征张量很好是二维，因此其特征张量与标签几乎总是分开的

In [1]:
import torch


In [3]:
from torch.utils.data import TensorDataset
a= torch.randn(500, 2, 3)
a

tensor([[[-0.8971,  0.1556, -0.5406],
         [-1.3456, -1.1140,  0.3868]],

        [[-0.2316,  0.3708,  0.1034],
         [ 0.6025,  1.0649, -0.0692]],

        [[-0.4352, -0.2196, -1.1732],
         [ 0.2054, -0.5285, -0.3931]],

        ...,

        [[-1.2930,  0.1117, -0.3044],
         [-0.0355, -0.3556,  1.1676]],

        [[-1.7796,  1.7336, -0.2559],
         [-3.0030,  0.8106,  0.3702]],

        [[-1.6833,  0.3935,  0.0084],
         [-0.4669, -1.5896,  0.7141]]])

In [4]:
b = torch.randn(500, 3, 4, 5)

In [5]:
c = torch.randn(500, 1)

In [6]:
# 被合并的对象在第一维度上相等
TensorDataset(a, b, c)

为什么要求在第一维度上相等呢？

    在 PyTorch 中，TensorDataset 是一个用于将多个张量组合成数据集的工具类。它要求传入的张量在第一维度上的值相等，这是因为 TensorDataset 的设计初衷是将不同的张量视为样本之间的不同特征。换句话说，每个传入的张量的第一维度代表样本的维度，而其他维度代表特征。

In [7]:
# DataLoader - 用来切割小批量的类

from torch.utils.data import DataLoader
data = TensorDataset(b, c)
for x in DataLoader(data):
    print(x)
    break


[tensor([[[[ 0.6054, -0.8963, -0.3871, -1.5956,  0.0473],
          [ 0.5799,  0.5379,  0.2733,  0.3890, -2.2284],
          [ 0.1813, -0.1851, -0.3398, -0.1406, -1.6551],
          [-0.8162,  0.5725,  0.6526, -0.7425,  0.0506]],

         [[ 0.8057,  0.3806, -1.6621,  0.0718,  0.4501],
          [ 0.8489,  1.0486, -0.6034, -0.4966,  0.6193],
          [-0.5315,  0.9355, -0.9673, -0.8007,  0.5893],
          [ 1.6153,  0.3694, -0.0655, -0.9130,  0.5840]],

         [[ 0.9566,  0.5806,  0.2932,  1.7895, -0.8922],
          [-0.3674,  1.5318, -1.2813, -0.4336,  0.1239],
          [ 0.4427,  0.5366, -1.5476,  0.7068, -0.8990],
          [ 0.6296,  1.4804, -0.4951,  0.8997,  0.7605]]]]), tensor([[-1.4819]])]


In [10]:
bs = 120
dataset = DataLoader(data
           , batch_size = bs
           , shuffle = True # 划分小批量之前请随机打乱我们的数据
           ,drop_last= True # 你要舍弃最后一个bathc吗？
           )

In [12]:
for i in dataset:
    print(i[0].shape)

torch.Size([120, 3, 4, 5])
torch.Size([120, 3, 4, 5])
torch.Size([120, 3, 4, 5])
torch.Size([120, 3, 4, 5])


In [13]:
len(dataset)    # 一共有多少个batch

4

In [1]:
len(dataset.dataset)    # 展示里面的全部数据

NameError: name 'dataset' is not defined